In [9]:
# Copyright (c) OpenMMLab. All rights reserved.
import os

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.parallel import MMDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from mmedit.apis import multi_gpu_test, set_random_seed, single_gpu_test
from mmedit.core.distributed_wrapper import DistributedDataParallelWrapper
from mmedit.datasets import build_dataloader, build_dataset
from mmedit.models import build_model
from mmedit.utils import setup_multi_processes

In [10]:
cfg_list = [
    r"D:\mmediting\configs\LLLR\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNN.py",
    r"D:\mmediting\configs\restorers\basicvsr_plusplus\basicvsr_plusplus_c64n7_8x1_600k_reds4.py",
    r"D:\mmediting\configs\LLLR\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNN.py",
    r"D:\mmediting\configs\LLLR\basicvsr_plusplus_c64n7_8x1_600k_reds4_2x_NLNN.py",
    r"D:\mmediting\configs\LLLR\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNoNoise_VimeoResume_4x.py"
]

PTHLIST = [
    
    # (r"D:\mmediting\Trained\BasicVSRPP_lowres_from_scratch\iter_95000.pth","BasicVSRPP_c64n7_FromScratch_95000iter",cfg_list[1]),
    # (r"D:\mmediting\work_dirs\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNN\iter_297000.pth","BasicVSRPP_c64n7_ResumeFrom300K_LLNN_297000iter",cfg_list[0]),
    # (r"D:\mmediting\Trained\BasicVSRPP_lowres_Resume\iter_601800.pth","BasicVSRPP_c64n7_ResumeFrom600K_601800iter",cfg_list[1]),
    # (r"D:\pretrained\basicvsr_plusplus_c64n7_8x1_300k_vimeo90k_bi_20210305-4ef437e2.pth","BasicVSRPP_c64n7_VimeoPretrained_300000iter",cfg_list[1]),
    # (r"D:\pretrained\basicvsr_plusplus_c64n7_8x1_600k_reds4_20210217-db622b2f.pth","BasicVSRPP_c64n7_REDSPretrained_600000iter",cfg_list[1]),
    # (r"D:\mmediting\Trained\BasicVSRPP_LLNN_vimeo_reume\iter_338000.pth","BasicVSRPP_c64n7_VimeoResume_338000",cfg_list[1])
    # (r"D:\mmediting\Trained\basicvsr_plusplus_c64n7_8x1_600k_reds4_2x_NLNN\iter_140000.pth","BasicVSRPP_c64n7_2X_FromScratch_140000iter",cfg_list[2]),
    # (r"D:\mmediting\Trained\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNoNoise_BlueCrystal\iter_410000.pth","BasicVSRPP_c64n7_4X_VimeoResume_LLNN_400000iter",cfg_list[3]),
    # (r"D:\mmediting\Trained\basicvsr_plusplus_c64n7_8x1_600k_reds4_NLNN\iter_20000.pth","BasicVSRPP_c64n7_4X_NLNN_20000iter",cfg_list[2]),
    # (r"D:\mmediting\work_dirs\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNN\iter_20000.pth","BasicVSRPP_c64n7_4X_LLNN_20000iter",cfg_list[2]),
    # (r"D:\mmediting\work_dirs\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLWN\iter_20000.pth","BasicVSRPP_c64n7_4X_LLWN_20000iter",cfg_list[2]),
    # (r"D:\mmediting\work_dirs\basicvsr_plusplus_c64n7_8x1_600k_reds4_NLWN\iter_20000.pth","BasicVSRPP_c64n7_4X_NLWN_20000iter",cfg_list[2])
    (r"D:\mmediting\Trained\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNoNoise_BlueCrystal\iter_490000.pth","BasicVSRPP_c64n7_4X_LLNN_490000",cfg_list[2])
]

In [11]:
def test(cfg_path,checkpoint,outpath,imgpath,dirpath):
    cfg =Config.fromfile(cfg_path)
    dataset = build_dataset(cfg.data.test)
    loader_cfg = {
        **dict((k, cfg.data[k]) for k in ['workers_per_gpu'] if k in cfg.data),
        **dict(
            samples_per_gpu=1,
            drop_last=False,
            shuffle=False,
            dist=False),
        **cfg.data.get('test_dataloader', {})
    }
    data_loader = build_dataloader(dataset,**loader_cfg)
    model = build_model(cfg.model,train_cfg=None,test_cfg=cfg.test_cfg)
    _ = load_checkpoint(model,checkpoint,map_location="cpu")
    model = MMDataParallel(model,device_ids=[0])
    outputs = single_gpu_test(
        model,
        data_loader,
        save_path=dirpath,
        save_image=imgpath,
    )


    print('')
    # print metrics
    stats = dataset.evaluate(outputs)
    for stat in stats:
        print('Eval-{}: {}'.format(stat, stats[stat]))    

    # save result pickle
    print('writing results to {}'.format(outpath))
    mmcv.dump(outputs, outpath)




In [12]:
for pth in PTHLIST:
    outpath = "D:\\Test_Result\\" + pth[1] + "\\result.json"
    dirpath = "D:\\Test_Result\\" + pth[1]
    imgpath = "D:\\Test_Result\\" + pth[1] + "\\Image"
    test(pth[2],pth[0],outpath,imgpath,dirpath)
    

2022-08-11 12:36:12,622 - mmedit - INFO - load checkpoint from http path: https://download.openmmlab.com/mmediting/restorers/basicvsr/spynet_20210409-c6c1bd09.pth


load checkpoint from local path: D:\mmediting\Trained\basicvsr_plusplus_c64n7_8x1_600k_reds4_LLNoNoise_BlueCrystal\iter_490000.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 0.0 task/s, elapsed: 347s, ETA:     0s
Eval-PSNR: 21.94535828316331
Eval-SSIM: 0.7465269445619283
writing results to D:\Test_Result\BasicVSRPP_c64n7_4X_LLNN_490000\result.json
